# How to stream arbitrary nested content

The most common use case for streaming from inside a node is to stream LLM tokens, but you may have other long-running streaming functions you wish to render for the user. While individual nodes in LangGraph cannot return generators (since they are executed to completion for each [superstep](https://langchain-ai.github.io/langgraph/concepts/low_level)), we can still stream content from arbitrary custom functions from within a node.

We do so in two ways:
* using graph's `.stream` / `.astream` methods with `stream_mode="custom"`
* using [RunnableLambda](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html#langchain_core.runnables.base.RunnableLambda) and emitting custom events using [adispatch_custom_events](https://python.langchain.com/docs/how_to/callbacks_custom_events/).

Below is a simple toy example that shows both.

## Setup

First, let's install our required packages

In [1]:
%%capture --no-stderr
%pip install -U langgraph

<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>    

## Stream nested content using `.stream / .astream`

### Define the graph

In [ ]:
from langchain_core.messages import AIMessage
from langgraph.graph import START, StateGraph, MessagesState, END
from langgraph.types import StreamWriter

async def my_generator(state: MessagesState):
    messages = [
        "Four",
        "score",
        "and",
        "seven",
        "years",
        "ago",
        "our",
        "fathers",
        "...",
    ]
    for message in messages:
        yield message


async def my_node(
    state: MessagesState, 
    writer: StreamWriter  # <-- provide StreamWriter to write chunks to be streamed
):
    messages = []
    async for message in my_generator(state):
        # write the chunk to be streamed using stream_mode=custom 
        writer(message)
        messages.append(message)

    return {"messages": [AIMessage(content=" ".join(messages))]}

# Define a new graph
workflow = StateGraph(MessagesState)

workflow.add_node("model", my_node)
workflow.add_edge(START, "model")
workflow.add_edge("model", END)

app = workflow.compile()

### Stream content

In [4]:
from langchain_core.messages import HumanMessage

inputs = [HumanMessage(content="What are you thinking about?")]
async for chunk in app.astream({"messages": inputs}, stream_mode="custom"):
    print(chunk, end="|", flush=True)

Four|score|and|seven|years|ago|our|fathers|...|

## Stream nested content using `.astream_events`

If you are already using graph's `.astream_events` method in your workflow, you can also stream arbitrary nested content using `RunnableLambda` and `adispatch_custom_event`

<div class="admonition warning">
    <p class="admonition-title">ASYNC IN PYTHON<=3.10</p>
    <p>

LangChain cannot automatically propagate configuration, including callbacks necessary for `astream_events()`, to child runnables if you are running async code in python<=3.10. This is a common reason why you may fail to see events being emitted from custom runnables or tools.

If you are running python<=3.10, you will need to manually propagate the `RunnableConfig` object to the child runnable in async environments. For an example of how to manually propagate the config, see the implementation of the `RunnableLambda` below.

If you are running python>=3.11, the RunnableConfig will automatically propagate to child runnables in async environment. However, it is still a good idea to propagate the `RunnableConfig` manually if your code may run in other Python versions.
    </p>
</div>

### Define the graph

In [5]:
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.callbacks.manager import adispatch_custom_event

@RunnableLambda
async def my_generator(
    state: MessagesState,
    config: RunnableConfig  # <-- propagate config
):
    messages = [
        "Four",
        "score",
        "and",
        "seven",
        "years",
        "ago",
        "our",
        "fathers",
        "...",
    ]
    for message in messages:
        await adispatch_custom_event(
            "my_generator",
            {"chunk": message},
            config=config  # <-- propagate config
        )
        yield message


async def my_node(state: MessagesState, config: RunnableConfig):
    messages = []
    async for message in my_generator.astream(state, config):
        messages.append(message)
    return {"messages": [AIMessage(content=" ".join(messages))]}

# Define a new graph
workflow = StateGraph(MessagesState)

workflow.add_node("model", my_node)
workflow.add_edge(START, "model")
workflow.add_edge("model", END)

app = workflow.compile()

### Stream content

In [6]:
from langchain_core.messages import HumanMessage

inputs = [HumanMessage(content="What are you thinking about?")]
async for event in app.astream_events({"messages": inputs}, version="v2"):
    tags = event.get("tags", [])
    if event["event"] == "on_custom_event" and event["name"] == "my_generator":
        data = event["data"]
        if data:
            print(data["chunk"], end="|", flush=True)

Four|score|and|seven|years|ago|our|fathers|...|